In [ ]:
Dependent variable - Price
All other - Independent variables

we will use Supervised techniques since the dataset is labelled

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

### read data from csv file

In [6]:
df_original = pd.read_csv("data/bengaluru_house_prices.csv")
df_original.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [7]:
df_original.shape

(13320, 9)

In [10]:
# drop certain columns
df_prices = df_original.drop(columns=["area_type","availability","society","balcony"])
df_prices.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


### Data Cleaning 

- check for null values
- removed unnecessary features
- check for outliers
- claen up values in columns - unstructured ones

In [11]:
# check for null values
df_prices.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [12]:
df_prices_final= df_prices.dropna()
df_prices_final.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [13]:
df_prices_final.shape

(13246, 5)

In [15]:
# check for all unique values
df_prices_final["size"].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

it contains duplicate values as 4 bedroom is same as 4 BHK and other dupliacte values

need a numeric column - so create a new column

In [16]:
df_prices_final["bhk"] = df_prices_final["size"].apply(lambda x: int(x.split(" ")[0]))

<ipython-input-16-7b037e8eaba4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices_final["bhk"] = df_prices_final["size"].apply(lambda x: int(x.split(" ")[0]))


In [17]:
df_prices_final.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [18]:
df_prices_final["bhk"].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18])

some values like bedroom = 43 looks unrealistic, so we will check the records for outliers

#### outliers and unstructured data

In [19]:
df_prices_final[df_prices_final.bhk > 20]

,location,size,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


second record with 43 bedroom and just 2400 square ft does not seem correct. we need to clean up these errors in the data.
first check the unique values in total_sqrt and variation in the data

In [20]:
df_prices_final["total_sqft"].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

#### Explore total_sqft feature

there are some values which are not the abslute number but the range of sqrt ft. we need to convert it to a number. We can take an average of the numbers
first check if the value is a float or not. creaye afunction and apply it to the column

In [21]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [29]:
unstr_data = df_prices_final[~df_prices_final.total_sqft.apply(is_float)] # need values where False so, ~
unstr_data["total_sqft"].unique()

array(['2100 - 2850', '3067 - 8156', '1042 - 1105', '1145 - 1340',
       '1015 - 1540', '34.46Sq. Meter', '1195 - 1440', '4125Perch',
       '1120 - 1145', '3090 - 5002', '1160 - 1195', '1000Sq. Meter',
       '1115 - 1130', '1100Sq. Yards', '520 - 645', '1000 - 1285',
       '650 - 665', '633 - 666', '5.31Acres', '30Acres', '1445 - 1455',
       '884 - 1116', '850 - 1093', '716Sq. Meter', '547.34 - 827.31',
       '580 - 650', '3425 - 3435', '1804 - 2273', '3630 - 3800',
       '4000 - 5249', '1500Sq. Meter', '142.61Sq. Meter', '1574Sq. Yards',
       '1250 - 1305', '670 - 980', '1005.03 - 1252.49', '1004 - 1204',
       '361.33Sq. Yards', '645 - 936', '2710 - 3360', '2830 - 2882',
       '596 - 804', '1255 - 1863', '1300 - 1405', '117Sq. Yards',
       '934 - 1437', '980 - 1030', '2249.81 - 4112.19', '1070 - 1315',
       '3040Sq. Meter', '500Sq. Yards', '2806 - 3019', '613 - 648',
       '704 - 730', '1210 - 1477', '3369 - 3464', '1125 - 1500',
       '167Sq. Meter', '1076 - 1199',

In [51]:
unstr_data.head()

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2


there are 190 columns which are not float values-some are in range, some have meters/yards mentioned

In [35]:
def convert_sqft_to_num(x):
    tokens = x.split("-")
    if len(tokens) == 2:
        return (float(tokens[0]) + float(tokens[1])) / 2
    try:
        float(x)
    except:
        return None
    return x

In [36]:
convert_sqft_to_num('2100 - 2850')

2475.0

In [52]:
df_final = unstr_data.copy() # creates a deep copy
df_final["total_sqft"] = unstr_data["total_sqft"].apply(convert_sqft_to_num)
df_final.head()

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2475.0,4.0,186.000,4
122,Hebbal,4 BHK,5611.5,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1073.5,2.0,54.005,2
165,Sarjapur,2 BHK,1242.5,2.0,43.490,2
188,KR Puram,2 BHK,1277.5,2.0,56.800,2


### Feature Engineering

In the real estate market, price per square feet is very important. This feature will also help us do some outlier cleaning at a later stage.

In [53]:
df_final["price_per_sqft"] = (df_final["price"] * 100000) / df_final["total_sqft"]
df_final.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
30,Yelahanka,4 BHK,2475.0,4.0,186.000,4,7515.151515
122,Hebbal,4 BHK,5611.5,4.0,477.000,4,8500.400962
137,8th Phase JP Nagar,2 BHK,1073.5,2.0,54.005,2,5030.740568
165,Sarjapur,2 BHK,1242.5,2.0,43.490,2,3500.201207
188,KR Puram,2 BHK,1277.5,2.0,56.800,2,4446.183953
